In [83]:
# !pip install -q pybotters numpy pandas
# !pip install -q python-dotenv


## Idea
* バブル相場ではどうせ価格は上がるんだから、一時的に下げた後のお得なところで買えば勝てるよね！

## 設計構想

* 直近N分の価格取得
  * 最低価格を計算


* 

***
## TODO
### 最優先
* volumeの自動取得
* order sizeの算出
* closing orderのsizeをpositionから算出

### 次点
* 複数銘柄対応

***

In [1]:
import pybotters
import pandas as pd
from pandas import DataFrame

In [2]:
class Exchange():
    """
    取引所と直接APIのやり取りを行う
    """
    def __init__(self, client):
        self.client = client
        pass
    
    async def get_klines(self, pair_symbol:str, interval:str='60', limit:str='5') -> pd.DataFrame:
        """
        /v5/market/kline
        ローソク足(kline)を取得
        """
        async with self.client.get(
            f"/v5/market/kline?category=linear&interval=60&symbol={pair_symbol}&interval={interval}&limit={limit}"
        ) as resp:
            content = await resp.json()

        klines = content['result']['list']
        klines = pd.DataFrame(klines)
        klines.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'turnover']

        return klines

    async def get_latest_price(self, pair_symbol:str):
        """"""""
        klines = await self.get_klines(pair_symbol, interval='1', limit='1')
        latest_price = float(klines['close'].values[-1])
        return latest_price

    async def get_order(self, pair_symbol:str):
        async with self.client.get(
            f"/v5/order/realtime?category=linear&symbol={pair_symbol}&limit=50"  # max limit == 50
        ) as resp:
            content = await resp.json()
        orders = content['result']['list']
        # print(f'exchange.get_order: {orders}')
        return orders


    async def get_symbol_info(self):
        """銘柄の情報を取得"""
        pass

    async def get_position(self):
        async with self.client.get("/v5/position/list?category=linear&settleCoin=USDT") as resp:
            content = await resp.json()
            symbol_list = [{'symbol': json['symbol'], 'side': json['side'], 'size': json['size']} for json in content['result']['list']]
            return symbol_list

    async def set_leverage(self, pair_symbol:str):
        """
        https://bybit-exchange.github.io/docs/v5/position/leverage
        """

        data = {
            'category': "linear", 
            'symbol': pair_symbol,
            'buyLeverage': self.leverage,
            'sellLeverage': self.leverage,
        }
        async with self.client.post("/v5/position/set-leverage", data=data) as resp:
            content = await resp.json()
            print('[exchange]-set_leverage:', content)
            return content['retMsg']
        pass

    async def create_order(self, pair_symbol, qty, side='Buy', price=None, reduce_only=False):
        data = data={
                'category': "linear",
                'symbol': pair_symbol,
                'side': side,
                'orderType': 'Limit',
                'price': price,
                'qty': qty,
                'reduceOnly': reduce_only,
        }
        # print(data)
        async with self.client.post("/v5/order/create", data=data) as resp:
            content = await resp.json()
            print(content)
            return content['retMsg']

    async def cancel_all_orders(self, pair_symbol):
        data = data={'category': "linear", 'symbol': pair_symbol}
        async with self.client.post("/v5/order/cancel-all", data=data) as resp:
            content = await resp.json()
            print(content)
            return content['retMsg']

class TradingStrategy():
    """
    ロジックをもとに売買に関わるシグナルを作成する
    """
    def __init__(self, exchange: Exchange, trading_volume: float=10, rate_of_drop:float=0.2, rate_of_pump:float=0.2, leverage: int=5):
        self.exchange = exchange
        self.trading_volume = trading_volume
        self.rate_of_drop = rate_of_drop
        self.rate_of_pump = rate_of_pump
        self.leverage = leverage
    
    async def initalize_setting(self, pair_symbol: str):
        # set leverage

        # cancel all existing orders
        ret_msg = await self.exchange.cancel_all_orders(pair_symbol)
        print('[TradingStrategy]-initalize_setting:', ret_msg)
        pass

    def calc_position_size(self):
        pass

    async def update_order(self):
        """
        update existing order
        """
        pass

    async def get_open_order(self, pair_symbol:str):
        """
        get existing order
        """
        orders = await self.exchange.get_order(pair_symbol)
        return orders


    async def calc_buy_price(self, pair_symbol:str, interval:str=60) -> float:
        klines:DataFrame = await self.exchange.get_klines(pair_symbol=pair_symbol, interval=interval, limit='5')
        high:float = klines['high'].astype(float).values
        highest_price:float = high.max()
        buy_price:float = highest_price * (1 - self.rate_of_drop)
        # レバレッジ部分は追加実装する必要あり
        return buy_price

    async def calc_sell_price(self, pair_symbol:str, interval:str=60) -> float:
        klines:DataFrame = await self.exchange.get_klines(pair_symbol=pair_symbol, interval=interval, limit='5')
        low:float = klines['low'].astype(float).values
        lowest_price:float = low.min()
        sell_price:float = lowest_price * (1 + self.rate_of_pump)
        return sell_price

    async def create_opening_order(self, pair_symbol) -> None:
        buy_price = await self.calc_buy_price(pair_symbol)
        ret_msg = await self.exchange.create_order(pair_symbol=pair_symbol, qty='0.01', side='Buy', price=str(int(buy_price)), reduce_only=False)
        print(f"Open position: {ret_msg}")

    async def create_closing_order(self, pair_symbol) -> None:
        sell_price = await self.calc_sell_price(pair_symbol)
        ret_msg = await self.exchange.create_order(pair_symbol=pair_symbol, qty='0.01', side='Sell', price=str(int(sell_price)), reduce_only=True)
        print(f"Close position: {ret_msg}")

class KaitenBot():
    """
    ロジックに沿ってトレードを行う最上位クラス
    """
    def __init__(self, client, strategy_params:dict):
        self.exchange = Exchange(client)
        self.trading_strategy = TradingStrategy(
            self.exchange,
            strategy_params['trading_volume'],
            strategy_params['rate_of_drop'],
            strategy_params['rate_of_pump'],
            strategy_params['leverage']
        )
    
    async def run(self):
        pair_symbol = 'ETHUSDT'

        # 0. cancel all orders
        await self.trading_strategy.initalize_setting(pair_symbol)

        # 1. check position
        positions = await self.exchange.get_position()
        current_position = None
        for position in positions:
            if position['symbol'] == pair_symbol:
                current_position = position
                print('current_position:', current_position)
                break

        # order = await self.trading_strategy.get_open_order(pair_symbol)
        # if order:
        #     print(f'There is {len(order)} orders in {pair_symbol}')
        # else:
        #     print(f'There is no orders in {pair_symbol}')

        # 2. create_order
        if current_position is None:
            # ポジションがない場合：新規ポジションを開く
            await self.trading_strategy.create_opening_order(pair_symbol)
        else:
            # ポジションがある場合：既存のポジションをクローズする
            await self.trading_strategy.create_closing_order(pair_symbol)

        order = await self.trading_strategy.get_open_order(pair_symbol)
        if order:
            print(f'There is {len(order)} orders in {pair_symbol}')
        else:
            print(f'There is no orders in {pair_symbol}')



In [5]:
import os
from dotenv import load_dotenv

load_dotenv('.env')

apis = {
    'bybit': [os.getenv('API_KEY'), os.getenv('API_SECRET')]
}

strategy_parmas = {
    'trading_volume':  float(os.getenv('volume')), 
    'leverage': int(os.getenv('leverage')),
    'rate_of_drop': float(os.getenv('rate_of_drop')),
    'rate_of_pump': float(os.getenv('rate_of_pump'))
}

print('strategy_parmas:', strategy_parmas, '\n')

async with pybotters.Client(apis=apis, base_url='https://api.bybit.com') as client:
    kaiten_bot = KaitenBot(client, strategy_parmas)
    await kaiten_bot.run()


strategy_parmas: {'trading_volume': 10.0, 'leverage': 5, 'rate_of_drop': 0.12, 'rate_of_pump': 0.07} 

{'retCode': 0, 'retMsg': 'OK', 'result': {'list': [{'orderId': '5c513c4c-a11d-4396-be06-86b62bb461d8', 'orderLinkId': ''}], 'success': '1'}, 'retExtInfo': {}, 'time': 1709794517936}
[TradingStrategy]-initalize_setting: OK
current_position: {'symbol': 'ETHUSDT', 'side': 'Buy', 'size': '0.01'}
{'retCode': 0, 'retMsg': 'OK', 'result': {'orderId': '60f63a6d-6f02-421e-9dec-ed23ea8fb3e7', 'orderLinkId': ''}, 'retExtInfo': {}, 'time': 1709794518285}
Close position: OK
There is 1 orders in ETHUSDT


## 参考

[バブル相場用の回転ボットのコンセプトとQuantZoneロジック(追記あり)](https://note.com/hht/n/n63022edc4610)

* 「レバレッジL倍で、直近N分高値から、R %価格が下がった位置にエントリー指値」
* 「直近N分安値から、R %価格が上がったら位置に決済指値」